# 1.0 - IMPORTS

## 1.1 - LIBRARIES

In [30]:
import gdown
import sqlite3
import pandas as pd
import numpy as np

## 1.2 - LOAD DATA

In [16]:
# URL do arquivo no Google Drive
url = "https://drive.google.com/uc?id=1HbqPT-GehWmmXnuXegbxx5cnjiXionvP"

# Caminho local onde você deseja salvar o arquivo (substitua pelo seu diretório)
output_path = "../data/dataset.db"  # Altere aqui

# Realiza o download
gdown.download(url, output_path, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1HbqPT-GehWmmXnuXegbxx5cnjiXionvP
To: /home/erickvieira/Documentos/repos/flight_project/data/dataset.db
100%|██████████| 12.6M/12.6M [00:01<00:00, 11.0MB/s]


'../data/dataset.db'

In [17]:
db_path = "../data/dataset.db"

# Conectar ao banco de dados
conn = sqlite3.connect(db_path)

# Criar o cursor para consultas
cursor = conn.cursor()

# Realizar a consulta
query = "SELECT * FROM flight_activity fa LEFT JOIN flight_loyalty_history flh ON fa.loyalty_number = flh.loyalty_number"
df_raw = pd.read_sql_query(query, conn)

# Fechar a conexão
conn.close()

## 1.3 - HELPER FUNCTIONS

# 2.0 - DATA DESCRIPTION

In [18]:
df2 = df_raw.copy()

## 2.1 - DATA DIMENSIONS

In [24]:
print('Numero de colunas: {}'.format(df2.shape[1]))
print('Numero de linhas: {}'.format(df2.shape[0]))
print('\nQuantidade de dados unicos para cada feature:')
df2.apply(lambda x: x.nunique())

Numero de colunas: 26
Numero de linhas: 405624

Quantidade de dados unicos para cada feature:


loyalty_number                 16737
year                               2
month                             12
flights_booked                    22
flights_with_companions           12
total_flights                     33
distance                        4746
points_accumulated              1549
points_redeemed                  587
dollar_cost_points_redeemed       49
loyalty_number                 16737
country                            1
province                          11
city                              29
postal_code                       55
gender                             2
education                          5
salary                          5890
marital_status                     3
loyalty_card                       3
clv                             7984
enrollment_type                    2
enrollment_year                    7
enrollment_month                  12
cancellation_year                  6
cancellation_month                12
dtype: int64

## 2.2 - RENAME COLUNS

In [25]:
df2.columns

Index(['loyalty_number', 'year', 'month', 'flights_booked',
       'flights_with_companions', 'total_flights', 'distance',
       'points_accumulated', 'points_redeemed', 'dollar_cost_points_redeemed',
       'loyalty_number', 'country', 'province', 'city', 'postal_code',
       'gender', 'education', 'salary', 'marital_status', 'loyalty_card',
       'clv', 'enrollment_type', 'enrollment_year', 'enrollment_month',
       'cancellation_year', 'cancellation_month'],
      dtype='object')

## 2.3 - DATA TYPES

In [26]:
df2.dtypes

loyalty_number                   int64
year                             int64
month                            int64
flights_booked                   int64
flights_with_companions          int64
total_flights                    int64
distance                         int64
points_accumulated             float64
points_redeemed                  int64
dollar_cost_points_redeemed      int64
loyalty_number                   int64
country                         object
province                        object
city                            object
postal_code                     object
gender                          object
education                       object
salary                         float64
marital_status                  object
loyalty_card                    object
clv                            float64
enrollment_type                 object
enrollment_year                  int64
enrollment_month                 int64
cancellation_year              float64
cancellation_month       

## 2.4 - NA VOLUMETRY

In [37]:
pd.DataFrame(np.round(df2.isna().mean()*100))

,0
loyalty_number,0.0
year,0.0
month,0.0
flights_booked,0.0
flights_with_companions,0.0
total_flights,0.0
distance,0.0
points_accumulated,0.0
points_redeemed,0.0
dollar_cost_points_redeemed,0.0
